In [1]:
import scanpy as sc
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import random

PROJECT_DIR = '/sc/arion/projects/CommonMind/collin/PSYCHAD_GRN'
os.chdir(PROJECT_DIR)

In [4]:
# Create directory for preprocessing 

if not os.path.exists(os.path.join(PROJECT_DIR, 'data/preprocessed')):
    os.makedirs('data/preprocessed')
    os.makedirs('data/processed/h5ad')

In [2]:
# Step 1: Load expression and normalize
H5AD_FNAME = '/sc/arion/projects/psychAD/NPS-AD/freeze2_proc/240124_PsychAD_freeze3_RUSH_clean.h5ad'
adata=sc.read_h5ad(H5AD_FNAME)
print(adata.shape)

#filter for protein coding
gene_annot = pd.read_csv('/sc/arion/projects/psychAD/NPS-AD/freeze2_rc/h5ad_final/gene_annotation.tsv', sep='\t')
protein_coding = gene_annot[gene_annot['gene_type'] == 'protein_coding']
protein_coding_genes = protein_coding['gene_name'].tolist()

#filter for protein coding
adata = adata[:, adata.var.index.isin(protein_coding_genes)]
#NPS_AD.write_h5ad('/sc/arion/projects/CommonMind/collin/PsychAD/data/h5ad/PsychAD_GRN_freeze3_proteincoding.h5ad')
print(adata.shape)


(693682, 34890)
(693682, 17259)


In [5]:
# Step 2: Apply standard filters

# Filter cells and genes
sc.pp.filter_cells(adata, min_genes=200 )
sc.pp.filter_genes(adata, min_cells=10)
print(adata.shape)


# Filter cells based on mitochondrial genes
adata = adata[adata.obs['n_genes'] < 6000, :]
print(adata.shape)
adata = adata[adata.obs['percent_mito'] < 0.10, :]
print(adata.shape)

# Filter for age 
adata = adata[adata.obs['Age'] > 65, :]
print(adata.shape)


# Filter for BB
adata = adata[adata.obs['Source'].isin(['R', 'M'])]
print(adata.shape)

#save h5ad
adata.write_h5ad(os.path.join(PROJECT_DIR, 'data/preprocessed/NPS_AD_RUSH_freeze3_proteincoding.h5ad'))

/sc/arion/projects/roussp01a/collin/envs/scenicplus2/lib/python3.8/site-packages/scanpy/preprocessing/_simple.py:140: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  adata.obs['n_genes'] = number


(693682, 17259)
(633049, 17259)
(274565, 17259)


KeyError: 'Age'

In [8]:
adata.write_h5ad(os.path.join(PROJECT_DIR, 'data/preprocessed/NPS_AD_RUSH_freeze3_proteincoding.h5ad'))

In [31]:
adata = sc.read_h5ad(os.path.join(PROJECT_DIR, 'data/preprocessed/NPS_AD_RUSH_freeze3_proteincoding.h5ad'))
adata

AnnData object with n_obs × n_vars = 274565 × 17259
    obs: 'Channel', 'SubID', 'rep', 'poolID', 'round_num', 'prep', 'SubID_cs', 'HTO_n_cs', 'max_prob', 'doublet_prob', 'Source', 'n_genes', 'n_counts', 'percent_mito', 'passed_SubID', 'scale', 'G1/S', 'G2/M', 'cycle_diff', 'cycling', 'predicted_phase', 'female_score', 'male_score', 'gender_score', 'predicted_gender', 'mito_genes', 'mito_ribo', 'ribo_genes', 'apoptosis', 'leiden_labels_res30', 'doublet_score', 'pred_dbl', 'class', 'subclass', 'subtype', 'Astro', 'EN_L2_3_IT', 'EN_L3_5_IT_1', 'EN_L3_5_IT_2', 'EN_L3_5_IT_3', 'EN_L5_6_NP', 'EN_L5_ET', 'EN_L6B', 'EN_L6_CT', 'EN_L6_IT_1', 'EN_L6_IT_2', 'Endo', 'IN_ADARB2', 'IN_LAMP5_LHX6', 'IN_LAMP5_RELN', 'IN_PVALB', 'IN_PVALB_CHC', 'IN_SST', 'IN_SST_HGF', 'IN_SST_NPY', 'IN_VIP', 'Immune', 'Micro_PVM', 'OPC', 'Oligo', 'PC', 'RB', 'SMC', 'VLMC', 'projID', 'date', 'pool', 'flowcell', 'lane'
    var: 'gene_id', 'gene_name', 'gene_type', 'gene_chrom', 'gene_start', 'gene_end', 'n_cells', 'perc

In [34]:
# Step 3: Add in metadata for contrasts 

adata.obs['barcodekey'] = adata.obs.index
adata.obs

metadata = pd.read_csv('/sc/arion/projects/psychAD/NPS-AD/freeze2_proc/230316_PsychAD_metadata_clean.csv')
metadata = metadata[['AD_strict', 'dx', 'age', 'sex', 'race', 'Braak', 'dementia', 'SubID']]
metadata = pd.merge(metadata, adata.obs, on='SubID')
metadata

adata.obs = metadata
adata.obs

adata.obs.set_index('barcodekey', inplace=True)
adata

AnnData object with n_obs × n_vars = 274565 × 17259
    obs: 'AD_strict', 'dx', 'age', 'sex', 'race', 'Braak', 'dementia', 'SubID', 'Channel', 'rep', 'poolID', 'round_num', 'prep', 'SubID_cs', 'HTO_n_cs', 'max_prob', 'doublet_prob', 'Source', 'n_genes', 'n_counts', 'percent_mito', 'passed_SubID', 'scale', 'G1/S', 'G2/M', 'cycle_diff', 'cycling', 'predicted_phase', 'female_score', 'male_score', 'gender_score', 'predicted_gender', 'mito_genes', 'mito_ribo', 'ribo_genes', 'apoptosis', 'leiden_labels_res30', 'doublet_score', 'pred_dbl', 'class', 'subclass', 'subtype', 'Astro', 'EN_L2_3_IT', 'EN_L3_5_IT_1', 'EN_L3_5_IT_2', 'EN_L3_5_IT_3', 'EN_L5_6_NP', 'EN_L5_ET', 'EN_L6B', 'EN_L6_CT', 'EN_L6_IT_1', 'EN_L6_IT_2', 'Endo', 'IN_ADARB2', 'IN_LAMP5_LHX6', 'IN_LAMP5_RELN', 'IN_PVALB', 'IN_PVALB_CHC', 'IN_SST', 'IN_SST_HGF', 'IN_SST_NPY', 'IN_VIP', 'Immune', 'Micro_PVM', 'OPC', 'Oligo', 'PC', 'RB', 'SMC', 'VLMC', 'projID', 'date', 'pool', 'flowcell', 'lane'
    var: 'gene_id', 'gene_name', 'gene_t

In [45]:
#Filter for AD / CONTROL, HVG, and save 
def subset_for_HVG(adata):
    adata = adata[:, adata.var['highly_variable_features']]
    return adata

adata = adata[adata.obs['dx'].isin(['AD', 'CTRL'])]
adata = subset_for_HVG(adata)
adata.write_h5ad(os.path.join(PROJECT_DIR, 'data/preprocessed/NPS_AD_RUSH_freeze3_metadata_HVG.h5ad'))



View of AnnData object with n_obs × n_vars = 123075 × 17259
    obs: 'AD_strict', 'dx', 'age', 'sex', 'race', 'Braak', 'dementia', 'SubID', 'Channel', 'rep', 'poolID', 'round_num', 'prep', 'SubID_cs', 'HTO_n_cs', 'max_prob', 'doublet_prob', 'Source', 'n_genes', 'n_counts', 'percent_mito', 'passed_SubID', 'scale', 'G1/S', 'G2/M', 'cycle_diff', 'cycling', 'predicted_phase', 'female_score', 'male_score', 'gender_score', 'predicted_gender', 'mito_genes', 'mito_ribo', 'ribo_genes', 'apoptosis', 'leiden_labels_res30', 'doublet_score', 'pred_dbl', 'class', 'subclass', 'subtype', 'Astro', 'EN_L2_3_IT', 'EN_L3_5_IT_1', 'EN_L3_5_IT_2', 'EN_L3_5_IT_3', 'EN_L5_6_NP', 'EN_L5_ET', 'EN_L6B', 'EN_L6_CT', 'EN_L6_IT_1', 'EN_L6_IT_2', 'Endo', 'IN_ADARB2', 'IN_LAMP5_LHX6', 'IN_LAMP5_RELN', 'IN_PVALB', 'IN_PVALB_CHC', 'IN_SST', 'IN_SST_HGF', 'IN_SST_NPY', 'IN_VIP', 'Immune', 'Micro_PVM', 'OPC', 'Oligo', 'PC', 'RB', 'SMC', 'VLMC', 'projID', 'date', 'pool', 'flowcell', 'lane'
    var: 'gene_id', 'gene_name',

In [2]:
#Define function to create directory

def create_GRN_dir(RESULTS_FNAME, h5ad):
    #Check if directory exists, if not create it
    directory_path = os.path.join(RESULTS_FNAME)
    if not os.path.exists(directory_path):
        os.makedirs(directory_path)
        os.makedirs(os.path.join(directory_path, 'csv'))
        os.makedirs(os.path.join(directory_path, 'h5ad'))
        os.makedirs(os.path.join(directory_path, 'regulons'))
        os.makedirs(os.path.join(directory_path, 'adj'))
        os.makedirs(os.path.join(directory_path, 'aucell'))
    
    #Save h5ad
    h5ad.write_h5ad(os.path.join(directory_path, 'h5ad', 'NPS_AD_RUSH_freeze3_original_metadata_HVG.h5ad'))
    h5ad.to_df().to_csv(os.path.join(directory_path, 'csv', 'NPS_AD_RUSH_freeze3_original_metadata_HVG.csv'))


In [54]:
adata = sc.read_h5ad(os.path.join(PROJECT_DIR, 'data/preprocessed/NPS_AD_RUSH_freeze3_metadata_HVG.h5ad'))
adata

AnnData object with n_obs × n_vars = 123075 × 5999
    obs: 'AD_strict', 'dx', 'age', 'sex', 'race', 'Braak', 'dementia', 'SubID', 'Channel', 'rep', 'poolID', 'round_num', 'prep', 'SubID_cs', 'HTO_n_cs', 'max_prob', 'doublet_prob', 'Source', 'n_genes', 'n_counts', 'percent_mito', 'passed_SubID', 'scale', 'G1/S', 'G2/M', 'cycle_diff', 'cycling', 'predicted_phase', 'female_score', 'male_score', 'gender_score', 'predicted_gender', 'mito_genes', 'mito_ribo', 'ribo_genes', 'apoptosis', 'leiden_labels_res30', 'doublet_score', 'pred_dbl', 'class', 'subclass', 'subtype', 'Astro', 'EN_L2_3_IT', 'EN_L3_5_IT_1', 'EN_L3_5_IT_2', 'EN_L3_5_IT_3', 'EN_L5_6_NP', 'EN_L5_ET', 'EN_L6B', 'EN_L6_CT', 'EN_L6_IT_1', 'EN_L6_IT_2', 'Endo', 'IN_ADARB2', 'IN_LAMP5_LHX6', 'IN_LAMP5_RELN', 'IN_PVALB', 'IN_PVALB_CHC', 'IN_SST', 'IN_SST_HGF', 'IN_SST_NPY', 'IN_VIP', 'Immune', 'Micro_PVM', 'OPC', 'Oligo', 'PC', 'RB', 'SMC', 'VLMC', 'projID', 'date', 'pool', 'flowcell', 'lane'
    var: 'gene_id', 'gene_name', 'gene_ty

In [55]:
#Split for RUSH (AD + CTRL)
adata = sc.read_h5ad(os.path.join(PROJECT_DIR, 'data/preprocessed/NPS_AD_RUSH_freeze3_metadata_HVG.h5ad'))
SAVE_DIR = os.path.join(PROJECT_DIR, 'GRN_splits/RUSH')
create_GRN_dir(SAVE_DIR, adata)


In [3]:
#Split for RUSH (AD)
adata = sc.read_h5ad(os.path.join(PROJECT_DIR, 'data/preprocessed/NPS_AD_RUSH_freeze3_metadata_HVG.h5ad'))
adata = adata[adata.obs['dx'].isin(['AD'])]
print(adata)
SAVE_DIR = os.path.join(PROJECT_DIR, 'GRN_splits/RUSH_AD')
create_GRN_dir(SAVE_DIR, adata)

#Split for RUSH (CTRL)
adata = sc.read_h5ad(os.path.join(PROJECT_DIR, 'data/preprocessed/NPS_AD_RUSH_freeze3_metadata_HVG.h5ad'))
adata = adata[adata.obs['dx'].isin(['CTRL'])]
print(adata)
SAVE_DIR = os.path.join(PROJECT_DIR, 'GRN_splits/RUSH_CTRL')    
create_GRN_dir(SAVE_DIR, adata)

print("Done!")


View of AnnData object with n_obs × n_vars = 123075 × 5999
    obs: 'AD_strict', 'dx', 'age', 'sex', 'race', 'Braak', 'dementia', 'SubID', 'Channel', 'rep', 'poolID', 'round_num', 'prep', 'SubID_cs', 'HTO_n_cs', 'max_prob', 'doublet_prob', 'Source', 'n_genes', 'n_counts', 'percent_mito', 'passed_SubID', 'scale', 'G1/S', 'G2/M', 'cycle_diff', 'cycling', 'predicted_phase', 'female_score', 'male_score', 'gender_score', 'predicted_gender', 'mito_genes', 'mito_ribo', 'ribo_genes', 'apoptosis', 'leiden_labels_res30', 'doublet_score', 'pred_dbl', 'class', 'subclass', 'subtype', 'Astro', 'EN_L2_3_IT', 'EN_L3_5_IT_1', 'EN_L3_5_IT_2', 'EN_L3_5_IT_3', 'EN_L5_6_NP', 'EN_L5_ET', 'EN_L6B', 'EN_L6_CT', 'EN_L6_IT_1', 'EN_L6_IT_2', 'Endo', 'IN_ADARB2', 'IN_LAMP5_LHX6', 'IN_LAMP5_RELN', 'IN_PVALB', 'IN_PVALB_CHC', 'IN_SST', 'IN_SST_HGF', 'IN_SST_NPY', 'IN_VIP', 'Immune', 'Micro_PVM', 'OPC', 'Oligo', 'PC', 'RB', 'SMC', 'VLMC', 'projID', 'date', 'pool', 'flowcell', 'lane'
    var: 'gene_id', 'gene_name', 

In [ ]:
import datetime
import random
import multiprocessing
import os
import pandas as pd

RESULTS_FOLDERNAME='/sc/arion/projects/CommonMind/collin/PSYCHAD_GRN/GRN_splits/RUSH'
#RESULTS_FOLDERNAME='/sc/arion/projects/CommonMind/collin/PsychAD/no_var_pilot/UNIVERSAL'
tf_list = '/sc/arion/projects/roussp01a/collin/scenic-kaiyi/data/db_f/allTFs_hg38.txt'


CSV_FNAME = os.path.join(RESULTS_FOLDERNAME, 'csv')
if os.path.exists(CSV_FNAME):
    for n in [111, 222, 333, 444, 555]:
        # Get path of csv file for each contrast ~ celltype ~ variable combination
        SEED = n
        print('Inferring GRN for UNIVERSAL run {} with seed {}'.format(n, SEED))
        csv_file_path = [os.path.join(CSV_FNAME, filename) for filename in os.listdir(CSV_FNAME) if os.path.isfile(os.path.join(CSV_FNAME, filename))][0]
        print(csv_file_path)
        
        ADJACENCIES_FNAME = csv_file_path.replace('.csv', '.adjacencies_run{}_seed_{}.tsv'.format(n, SEED))
        ADJACENCIES_FNAME = ADJACENCIES_FNAME.replace('/csv/', '/adj/').replace('_PsychAD_GRN', '')
        
        if os.path.exists(ADJACENCIES_FNAME):
            print('Already exists: {}'.format(ADJACENCIES_FNAME))
            continue
        print(ADJACENCIES_FNAME)
        
        # Inferring GRN
        cores = multiprocessing.cpu_count() - 2

        start_time = datetime.datetime.now()
        print(f"Start time: {start_time.strftime('%Y-%m-%d %H:%M:%S')}")

        # System call to arboreto_with_multiprocessing.py
        os.system(f"arboreto_with_multiprocessing.py {csv_file_path} {tf_list} --method grnboost2 --output {ADJACENCIES_FNAME} --num_workers {cores} --seed {SEED}")

        end_time = datetime.datetime.now()
        print(f"End time: {end_time.strftime('%Y-%m-%d %H:%M:%S')}")

        duration = (end_time - start_time).total_seconds() / 60
        print('Total runtime: {} minutes'.format(duration))

